# `Nuqleon.Linq.Expressions.Optimizers`

Provides optimizers for expression trees.

## Reference the library

### Option 1 - Use a local build

If you have built the library locally, run the following cell to load the latest build.

In [ ]:
#r "bin/Debug/net50/Nuqleon.Linq.Expressions.Optimizers.dll"

### Option 2 - Use NuGet packages

If you want to use the latest published package from NuGet, run the following cell.

In [ ]:
#r "nuget:Nuqleon.Linq.Expressions.Optimizers,*-*"

## (Optional) Attach a debugger

If you'd like to step through the source code of the library while running samples, run the following cell, and follow instructions to start a debugger (e.g. Visual Studio). Navigate to the source code of the library to set breakpoints.

In [ ]:
System.Diagnostics.Debugger.Launch();

## `ExpressionOptimizer`

The `ExpressionOptimizer` class is an expression tree visitor that rewrites an expression tree by performing various types of optimizations that can be configured by the user.

Optimizers can come in handy to reduce the size and evaluation complexity at runtime. For example, in the context of Reaqtor expression trees get serialized, sent across cients and services, get stored in databases, and parts of them get evaluated for many events. Therefore, it makes sense to optimize expressions in many ways:

* Reduce the size, to make I/O more efficient.
* Reduce runtime overheads, including:
  * CPU time to evaluate expressions;
  * Memory allocations.

As an example, consider a query like this:

```csharp
Task CreateWeatherSubscription(ClientContext ctx, string city)
{
    return ctx.GetObservable<WeatherInfo>(weatherUri).Where(x => x.City.ToLower() == city.ToLower()).SubscribeAsync(subUri, observer);
}
```

where we omitted some details. Upon submitting this query to the service, we'll inline the value of `city` in the query, resulting in a query of the following form:

```csharp
weather.Where(x => x.City.ToLower() == "Seattle".ToLower())
```

Note that this query would evaluate `"Seattle".ToLower()` for every weather event received on the stream. This has both CPU and memory costs. It'd be nice if the expression got optimized to:

```csharp
weather.Where(x => x.City.ToLower() == "seattle")
```

This, and much more, is what the expression optimizer can achieve. Let's have a look.

To create an optimizer instance, the constructor accepts two parameters:

* `ISemanticProvider` to specify a semantic provider that's consulted by the optimizer to make optimization decisions;
* `IEvaluatorFactory` to control the behavior of partial evaluation of subtrees.

We can have a first look at the optimizer by providing defaults for these parameters. In subsequent paragraphs of the notebook we'll get to the next level of detail.

In [ ]:
using System.Linq.Expressions;

var sem = new DefaultSemanticProvider();
var eval = new DefaultEvaluatorFactory();

var opt = new ExpressionOptimizer(sem, eval);

To illustrate optimizations, let's first craft an expression by hand.

In [ ]:
var expr = Expression.Add(Expression.Constant(1), Expression.Constant(2));

Console.WriteLine(expr);

(1 + 2)


Obviously, we can perform constant folding on this expression to reduce it to a single `ConstantExpression` node whose value is `3`. By running the optimizer's `Visit` method, we get exactly that result.

In [ ]:
var optimized = opt.Visit(expr);

Console.WriteLine(optimized);

3


What about a more complex expression that involves method calls and whatnot? An example is shown below, this time using `Expression<T>` language support to construct such an expression.

In [ ]:
Expression<Func<string>> f = () => "foobarqux".ToUpper().Substring(2, int.Parse("3"));

Console.WriteLine(f);

var optimized = opt.Visit(f);

Console.WriteLine(optimized);

() => "foobarqux".ToUpper().Substring(2, Parse("3"))


() => "foobarqux".ToUpper().Substring(2, 3)


This time, nothing happens, because the default semantic provider does not supply information about the purity of methods that enables it to perform evaluation of these subexpressions during optimization. To achieve this, we can zoom in to the semantic provider a tiny bit.

In [ ]:
var intParseMethod = typeof(int).GetMethod(nameof(int.Parse), new[] { typeof(string) });

Console.WriteLine(sem.IsPure(intParseMethod));

False


The optimizer talks to the semantic provider to ask questions like `IsPure(expr)` to check whether an expression is pure. There are many more questions it can ask, which we shown below by just dumping the interface's members.

In [ ]:
foreach (var m in typeof(ISemanticProvider).GetMethods().Select(m => m.Name).Distinct().OrderBy(m => m))
{
    Console.WriteLine(m);
}

AllBitsOne


AllBitsZero


AlwaysThrows


GetConstantValue


HasConstantValue


...

IsOne


IsPure


IsTrue


IsZero


NeverThrows


How can we teach the semantic provider that `int.Parse(string)` is pure so an optimizer can perform partial evaluation?

### Building a simple custom semantic provider

One option is to inherit from `DefaultSemanticProvider` and override the method.

In [ ]:
using System.Reflection;

class MySemanticProvider : DefaultSemanticProvider
{
    public override bool IsPure(MemberInfo member)
    {
        return base.IsPure(member) || member == typeof(int).GetMethod(nameof(int.Parse), new[] { typeof(string) });
    }
}

Later, we'll see how this can be made easier by building catalogs. For now, let's stick with this approach and create a new optimizer instance using the semantic provider shown above.

In [ ]:
var sem = new MySemanticProvider();
var eval = new DefaultEvaluatorFactory();

var opt = new ExpressionOptimizer(sem, eval);

If we run this optimizer over our expression, we get a different result.

In [ ]:
Console.WriteLine(f);

var optimized = opt.Visit(f);

Console.WriteLine(optimized);

() => "foobarqux".ToUpper().Substring(2, Parse("3"))


() => "foobarqux".ToUpper().Substring(2, 3)


Note that `int.Parse("3")` was evaluated to `3`.

> **Note:** The attentive reader will remark that `int.Parse(string)` is not pure, because it depends on the current culture. However, if an environment is configured in such a way that the result is predictable, e.g. the expression is optimized in the same environment where it is being evaluated, the optimization is valid. The extensible nature of semantic providers enables to make this type of choices.

It goes without saying that having to implement all of these purity checks can get cumbersome really fast. There are tons of members in the .NET Framework that have specific characteristics such as purity (but, as we will learn later, there are many other semantic questions). For example, we'd have to list `Substring` and `ToUpper` as pure. Luckily, there's a notion of catalogs.

### Using built-in catalogs

This library ships with a set of catalogs for commonly used semantic questions. An example is the `PureMemberCatalog`, which is shown below for the `System.String` type.

In [ ]:
var catalog = PureMemberCatalog.System.String;

foreach (var member in catalog)
{
    Console.WriteLine(member);
}

Void .ctor(Char[])


Void .ctor(Char[], Int32, Int32)


Void .ctor(Char, Int32)


System.String Empty


Int32 get_Length()


...

System.String Trim(Char[])


System.String TrimEnd(Char[])


System.String TrimStart(Char[])


Boolean op_Equality(System.String, System.String)


Boolean op_Inequality(System.String, System.String)


Built-in catalogs are organized by namespace and type, so you could find the pure members on e.g. `Regex` as follows:

In [ ]:
foreach (var member in PureMemberCatalog.System.Text.RegularExpressions.Regex)
{
    Console.WriteLine(member);
}

Void .ctor(System.String)


Void .ctor(System.String, System.Text.RegularExpressions.RegexOptions)


Void .ctor(System.String, System.Text.RegularExpressions.RegexOptions, System.TimeSpan)


System.TimeSpan get_MatchTimeout()


System.TimeSpan MatchTimeout


...

System.String Replace(System.String, System.String)


System.String Replace(System.String, System.String, Int32)


System.String Replace(System.String, System.String, Int32, Int32)


System.String ToString()


System.String Unescape(System.String)


In this context, purity means that the result of evaluating the member with its target instance (if any) and all of its arguments being constants will always produce the same result.

Member catalogs are further structured such that one can obtain a catalog for an entire namespace, with or without including child namespaces. Some examples are shown below:

In [ ]:
Console.WriteLine($"Pure member count in System = {PureMemberCatalog.System.AllThisNamespaceOnly.Count()}");
Console.WriteLine($"Pure member count on System.DateTimeOffset = {PureMemberCatalog.System.DateTimeOffset.Count()}");
Console.WriteLine($"Pure member count in System.Text.RegularExpressions = {PureMemberCatalog.System.Text.RegularExpressions.AllThisNamespaceOnly.Count()}");
Console.WriteLine($"Pure member count on System.Text.RegularExpressions.Regex = {PureMemberCatalog.System.Text.RegularExpressions.Regex.Count()}");
Console.WriteLine($"Pure member count in System.Collections and all child namespaces = {PureMemberCatalog.System.Collections.AllThisAndChildNamespaces.Count()}");
Console.WriteLine($"Total pure member count = {PureMemberCatalog.All.Count()}");

Pure member count in System = 943


Pure member count on System.DateTimeOffset = 68


Pure member count in System.Text.RegularExpressions = 74


Pure member count on System.Text.RegularExpressions.Regex = 36


Pure member count in System.Collections and all child namespaces = 5


Total pure member count = 1022


Member catalogs are immutable, but one can construct custom catalogs on top of existing ones, as shown below:

In [ ]:
var myCatalog = new MemberTable
{
    PureMemberCatalog.All,
    typeof(int).GetMethod(nameof(int.Parse), new[] { typeof(string) }),
    typeof(string).GetMethod(nameof(string.ToUpper), Type.EmptyTypes),
};

Note that we've added `ToUpper` ourselves, because the default catalog doesn't consider this method to be pure either, because it does depend on the current culture. However, the default catalog does contain `Substring`, so we don't have to worry about that.

Armed with this catalog, we can construct a `MetadataSemanticProvider` which inherits from `DefaultSemanticProvider` but provides properties that enable setting things such as the `PureMembers`.

In [ ]:
var sem = new MetadataSemanticProvider { PureMembers = myCatalog };
var eval = new DefaultEvaluatorFactory();

var opt = new ExpressionOptimizer(sem, eval);

When we try to optimize our expression now, we'd expect to get a different result.

In [ ]:
Console.WriteLine(f);

var optimized = opt.Visit(f);

Console.WriteLine(optimized);

() => "foobarqux".ToUpper().Substring(2, Parse("3"))


() => "OBA"


Indeed, the result is a single constant node containing the value `"OBA"` which is the result of evaluating `ToUpper`, `Substring`, and `Parse`.

But what if the attempt to partially evaluate an expression causes an exception to be thrown? An example is shown below.

In [ ]:
Expression<Func<string>> f = () => "foobarqux".ToUpper().Substring(10);

var optimized = opt.Visit(f);

Console.WriteLine(optimized);

() => throw(System.ArgumentOutOfRangeException: startIndex cannot be larger than length of string. (Parameter 'startIndex')
   at System.String.Substring(Int32 startIndex, Int32 length)
   at lambda_method310(Closure , Object , Object ))


This time, the expression gets rewritten to contain an `Expression.Throw` expression that will throw the exception that was encountered during partial evaluation at optimization time.

## A closer look at `ISemanticProvider`

Semantic providers are used by the optimizer to gather semantic information about expressions, types, members, values, etc. The `ISemanticProvider` interface represents all the capabilities of a semantic provider.

While custom implementations of this interface are possible, a good default choice for the semantic provider is `DefaultSemanticProvider` or `MetadataSemanticProvider`. The latter is more powerful because it supports specifying semantic information about .NET types and members, for example if a given type is immutable, or if a given member is a pure function. The library comes with various catalogs for commonly used types and members in the .NET Base Class Libraries, which can be used to construct a `MetadataSemanticProvider` as shown below.

In [ ]:
var msp = new MetadataSemanticProvider
{
    PureMembers = PureMemberCatalog.All,
    ConstParameters = ConstParameterCatalog.All,
    ImmutableTypes = ImmutableTypeCatalog.All
};

var opt = new ExpressionOptimizer(msp, new DefaultEvaluatorFactory());

void Example(Expression expr)
{
    Console.WriteLine(expr);

    var optimized = opt.Visit(expr);

    Console.WriteLine(optimized);
}

### Pure members

We've already looked at pure members before. Pure members are used to perform partial evaluation of nodes such as `MethodCallExpression`, for example `Math.Abs(n)` where `n` itself is pure.

In [ ]:
Example(Expression.Call(typeof(Math).GetMethod(nameof(Math.Abs), new[] { typeof(int) }), Expression.Constant(-2)));

Abs(-2)


2


### Constant parameters

Constant parameters are required for partial evaluation of a function if any of its parameters has a mutable type but the function doesn't perform any mutation, e.g. `string.Split(string, char[])` doesn't mutate the given `char[]`. This makes the following optimization work.

In [ ]:
Expression<Func<string, string[]>> f = s => s.Split(',', ';');

Example(f);

s => s.Split(new [] {,, ;})


s => s.Split(value(System.Char[]))


The allocation of the `char[]` can now be avoided for every evaluation of the expression at runtime, because a single constant `char[]` is used. This is safe because we know that `Split` only reads from the array and never mutates its contents. If we have another custom API that exhibits this behavior, we can add it to the catalog as well. Let's first define such a method.

In [ ]:
class Bar
{
    public static string Foo(string s, params int[] xs) => s + " = " + string.Join(",", xs);
}

Expression<Func<string>> f = () => Bar.Foo("qux", 1, 2, 3);

Example(f);

() => Foo("qux", new [] {1, 2, 3})


() => Foo("qux", new [] {1, 2, 3})


To add `Bar.Foo(int[])` to the catalog, we have to specify a pattern that indicates which parameter of `Foo` is to be treated as `const`. This is shown below:

In [ ]:
var constParameterTable = new ParameterTable { ConstParameterCatalog.All };

constParameterTable.Add<int[]>(xs => Bar.Foo("", xs));

msp = new MetadataSemanticProvider
{
    PureMembers = PureMemberCatalog.All,
    ConstParameters = constParameterTable,
    ImmutableTypes = ImmutableTypeCatalog.All
};

opt = new ExpressionOptimizer(msp, new DefaultEvaluatorFactory());

When we apply the optimizer this time, the `NewArrayInit` expression can get reduced to a constant as well.

In [ ]:
Example(f);

() => Foo("qux", new [] {1, 2, 3})


() => Foo("qux", value(System.Int32[]))


Note that if `Foo` itself would be marked as a pure member, the whole `Foo("qux", 1, 2, 3)` expression could be evaluated during optimization. Let's demonstrate this to show how all of these optimizations can "cascade".

In [ ]:
var pureMemberTable = new MemberTable
{
    PureMemberCatalog.All,
    typeof(Bar).GetMethod(nameof(Bar.Foo))
};

msp = new MetadataSemanticProvider
{
    PureMembers = pureMemberTable,
    ConstParameters = constParameterTable,
    ImmutableTypes = ImmutableTypeCatalog.All
};

opt = new ExpressionOptimizer(msp, new DefaultEvaluatorFactory());

When we apply the optimizer this time, the whole expression gets reduced to a constant.

In [ ]:
Example(f);

() => Foo("qux", new [] {1, 2, 3})


() => "qux = 1,2,3"


### Immutable types

Finally, checks for immutable types are used for various checks to ensure a member can't mutate the state of an object, e.g. `System.Tuple<T1, T2>` is immutable so it's safe to evaluate an instance of this type to a `Constant` such that subsequent calls can be made. Another well-known immutable type is `string`. For example, the reason that reducing `"foo".Substring(1).ToUpper()` to `"oo".ToUpper()` is because it's known that none of the members on `string` can cause mutation. Therefore, it's safe to have a `Constant` containing `"oo"` rather than evaluating `"foo".Substring(1)` every time to create a unique instance.

To demonstrate this principle, we can use a custom record type in C# 9.0.

In [ ]:
record Person(string Name, int Age);

A record is immutable, but the optimizer does have no notion of this. As such, if we try to construct an instance of `Person` given constant arguments, it does not know to optimize this to a constant. Let's show this below.

In [ ]:
Expression<Func<int>> f = () => new Person("Bart", 21).Age;

Example(f);

() => new Person("Bart", 21).Age


() => new Person("Bart", 21).Age


First, we can attempt to add the constructor of `Person` to the pure member catalog, as shown below:

In [ ]:
var pureMemberTable = new MemberTable
{
    PureMemberCatalog.All,
    typeof(Bar).GetMethod(nameof(Bar.Foo)),
    typeof(Person).GetConstructor(new[] { typeof(string), typeof(int) })
};

msp = new MetadataSemanticProvider
{
    PureMembers = pureMemberTable,
    ConstParameters = constParameterTable,
    ImmutableTypes = ImmutableTypeCatalog.All
};

opt = new ExpressionOptimizer(msp, new DefaultEvaluatorFactory());

However, when we try to optimize the expression, we still are out of luck.

In [ ]:
Example(f);

() => new Person("Bart", 21).Age


() => new Person("Bart", 21).Age


Even though the constructor is pure, the result of evaluating it would be a `ConstantExpression` containing a value of type `Person`. If `Person` were mutable, the optimization would be unsafe, because the shared instance could get mutated. We need to teach the semantic provider that `Person` is immutable, making it safe to share an instance.

In [ ]:
var immutableTypes = new TypeTable
{
    ImmutableTypeCatalog.All,
    typeof(Person)
};

msp = new MetadataSemanticProvider
{
    PureMembers = pureMemberTable,
    ConstParameters = constParameterTable,
    ImmutableTypes = immutableTypes
};

opt = new ExpressionOptimizer(msp, new DefaultEvaluatorFactory());

This time aorund, optimization is more fruitful.

In [ ]:
Example(f);

() => new Person("Bart", 21).Age


() => Person { Name = Bart, Age = 21 }.Age


This may be subtle, but the `ToString` of the expression tree's `ConstantExpression` node is showing the result of calling `ToString` on `Person`, which reads as `Person { Name = Bart, Age = 21 }`. Obivously, we could take this one step further and let the optimizer know that `Name` and `Age` properties are pure as well, meaning that when evaluated on a constant `Person` instance, they will always return the same result.

In [ ]:
pureMemberTable.Add(typeof(Person).GetProperty(nameof(Person.Name)));
pureMemberTable.Add(typeof(Person).GetProperty(nameof(Person.Age)));

msp = new MetadataSemanticProvider
{
    PureMembers = pureMemberTable,
    ConstParameters = constParameterTable,
    ImmutableTypes = immutableTypes
};

opt = new ExpressionOptimizer(msp, new DefaultEvaluatorFactory());

And finally, we end up with the whole expression reducing to `21`.

In [ ]:
Example(f);

() => new Person("Bart", 21).Age


() => 21


To show all of the optimizations combined, let's shown a more complex expression that involves an `InvocationExpression` of a `LambdaExpression` that prints a `Person` object.

In [ ]:
Expression<Func<Person, string>> toString = p => p.Name + " is " + p.Age.ToString();
Expression<Func<Person>> newPerson = () => new Person("Bart", 21);

var e = Expression.Invoke(toString, newPerson.Body);

Example(e);

Invoke(p => ((p.Name + " is ") + p.Age.ToString()), new Person("Bart", 21))


("Bart is " + 21.ToString())


Note that the optimizer got quite far in constant folding this fairly complex expression. The original expression looked like this:

```csharp
(p => (p.Name + " is ") + p.Age.ToString())(new Person("Bart", 21))
```

where the string concatenation was carried out in two steps and involving a boxing conversion for the age value.

First, it knew that evaluating `new Person("Bart", 21)` was safe to do, resulting in a `ConstantExpression`:

```csharp
(p => (p.Name + " is ") + p.Age.ToString())(c)
```

where `c` is a constant containing `Person { Name = "Bart", Age = 21 }`.

Next, this enabled inlining (also known as beta reduction) of the `Person` argument when invoking the `toString` lambda expression, so we ended up with:

```csharp
(c.Name + " is ") + c.Age.ToString()
```

Because `Name` and `Age` are pure, this got further rewritten into:

```csharp
("Bart" + " is ") + 21.ToString()
```

The string concatenation operator for two `string` arguments is also considered pure in the default pure members catalog, so more constant evaluation took place:

```csharp
"Bart is " + 21.ToString()
```

And this is where the optimization ended, because `int.ToString()` is culture-sensitive and therefore not considered pure.

## Evaluator factories

Evaluator factories are used to perform partial evaluation of an expression tree, e.g. for a node whose behavior is pure and whose children are constants. An example of such an expression is shown below.

In [ ]:
var expr = Expression.Add(Expression.Constant(DateTime.Now), Expression.Constant(TimeSpan.FromHours(1)));

Console.WriteLine(expr);

(3/5/2021 11:43:54 AM + 01:00:00)


This node is a `BinaryExpression` of type `Add` where the `Method` refers to `op_Addition(DateTime, TimeSpan)` on `System.DateTime`. The default pure member catalog contains this method. If we run this expression through the optimizer, the evaluator factory is used to get a delegate that can evaluate this `+` operation given two constant operands. To illustrate this behavior, we'll implement `IEvaluatorFactory`, or rather inherit from `DefaultEvaluatorFactory` to add some logging.

In [ ]:
class MyEvaluatorFactory : DefaultEvaluatorFactory
{
    public override Delegate GetEvaluator(MethodInfo method)
    {
        var res = base.GetEvaluator(method);

        Console.WriteLine($"Got evaluator for {method}.");

        return res;
    }
}

Using the metadata semantic provider and the custome valuator factory, we can construct an expression optimizer instance.

In [ ]:
var sem = new MetadataSemanticProvider { PureMembers = { PureMemberCatalog.All }, ImmutableTypes = { ImmutableTypeCatalog.All }, ConstParameters = { ConstParameterCatalog.All } };
var eval = new MyEvaluatorFactory();

var opt = new ExpressionOptimizer(sem, eval);

Let's now apply this optimizer to our expression and see the invocation to the evaluator.

In [ ]:
var optimized = opt.Visit(expr);

Console.WriteLine(optimized);

Got evaluator for System.DateTime op_Addition(System.DateTime, System.TimeSpan).


3/5/2021 12:43:54 PM


Note that the optimizer does not cache the delegates returned from the evaluator factory. This is done to avoid leaks, and one can perform caching within the evaluator factory instead. To show the lack of caching, we can apply the optimizer again to another expression.

In [ ]:
Expression<Func<DateTime, double, DateTime>> addHours = (dt, hours) => dt + TimeSpan.FromHours(hours);

var anotherExpr = Expression.Invoke(addHours, Expression.Constant(DateTime.Now), Expression.Constant(2.0));

var anotherOptimizedExpr = opt.Visit(anotherExpr);

Console.WriteLine(anotherOptimizedExpr);

Got evaluator for System.TimeSpan FromHours(Double).


Got evaluator for System.DateTime op_Addition(System.DateTime, System.TimeSpan).


3/5/2021 1:45:54 PM


One way to provide caching at the evaluator factory level would be to use `Nuqleon.Memory`'s memoization support. An example of how to compose these pieces is shown below.

In [ ]:
using System.Memory;

class MyMemoizingEvaluatorFactory : MyEvaluatorFactory, IClearable
{
    private readonly IMemoizedDelegate<Func<MemberInfo, Delegate>> _memoizedGetEvaluator;

    public MyMemoizingEvaluatorFactory(IMemoizationCacheFactory factory)
    {
        var mem = Memoizer.Create(factory);

        _memoizedGetEvaluator = mem.Memoize<MemberInfo, Delegate>(base.GetEvaluator);
    }

    public override Delegate GetEvaluator(MemberInfo member)
    {
        return _memoizedGetEvaluator.Delegate(member);
    }

    public void Clear()
    {
        _memoizedGetEvaluator.Cache.Clear();
    }
}

A real implementation would override a few more evaluator factory methods, but this suffices to demonstrate the effect by constructing a new optimizer that uses our memoizing evaluator factory. Also note that we're overriding `GetEvaluator(MemberInfo)` rather than the overload with `MethodInfo`. This is the entry-point method on the interface, so it allows for caching of evaluators of different member kinds (i.e. fields, properties, constructors, and methods).

In [ ]:
var sem = new MetadataSemanticProvider { PureMembers = { PureMemberCatalog.All }, ImmutableTypes = { ImmutableTypeCatalog.All }, ConstParameters = { ConstParameterCatalog.All } };

var cacheFactory = ConcurrentMemoizationCacheFactory.CreateLru(16);
var eval = new MyMemoizingEvaluatorFactory(cacheFactory);

var opt = new ExpressionOptimizer(sem, eval);

When we apply the optimizer to both expressions, we'll see the effects of caching.

In [ ]:
opt.Visit(expr);
opt.Visit(anotherExpr);

Got evaluator for System.DateTime op_Addition(System.DateTime, System.TimeSpan).


Got evaluator for System.TimeSpan FromHours(Double).


This time, we only see one call for `op_Addition`, as expected. Because our cache policy is set to LRU and to only contain 16 entries, we'll get a cap on the memory used.

> **Note:** Many uses of expression optimizers in Reaqtor stacks use unbounded caches during optimization of a whole bunch of expressions trees, e.g. when trying to compact the expressions in a query engine. At the end of the optimization pass, the caches are cleared or simply dropped and garbage collected. Either way, the design of decoupling the optimizer from aspects such as semantic providers and evaluator factories allows for a great deal of flexibility and separation of concerns.

## Customizing the `ExpressionOptimizer` by overriding `Visit` methods

The behavior of the expression visitor can be influenced by overriding various `Visit` methods as well. Three customization points are worth mentioning:

```csharp
protected virtual bool ShouldOptimize(Expression node);

protected virtual Expression VisitPreOptimize(Expression node);
protected virtual Expression VisitPostOptimize(Expression original, Expression optimized);
```

In case it's undesirable for certain nodes to get optimized, one can override `ShouldOptimize`. Returning `false` from this method will cause the optimizer to stop traversing the given expression. Examples include retaining the exact shape of a `Quote` expression, or preventing any optimization for nodes of a certain `Type`. For example, if a node could get partially evaluated to a constant of some type `T` which is not supported by some serializer that will run post optimization, one can avoid that such constants end up in the tree. Alternatively, one could override `VisitPostOptimize` and return the original expression if a rewrite was undesirable. This enables a specialized optimizer to "change its mind".

Rather than discussing all the possible ways these methods can be used, we'll just use them for logging in the example below. This also sheds some light on the optimizer's behavior.

In [ ]:
class LoggingExpressionOptimizer : ExpressionOptimizer
{
    private string _padLeft = "";

    public LoggingExpressionOptimizer(ISemanticProvider semanticProvider, IEvaluatorFactory evaluatorFactory)
        : base(semanticProvider, evaluatorFactory)
    {
    }

    public override Expression Visit(Expression node)
    {
        Console.WriteLine($"{_padLeft}{nameof(Visit)}({node}) \r\n{_padLeft}{{");
        Indent();
        var res = base.Visit(node);
        Outdent();
        Console.WriteLine($"{_padLeft}}} = {res}");
        return res;
    }

    protected override bool ShouldOptimize(Expression node)
    {
        var res = base.ShouldOptimize(node);
        Console.WriteLine($"{_padLeft}{nameof(ShouldOptimize)}({node}) = {res}");
        return res;
    }

    protected override Expression VisitPreOptimize(Expression node)
    {
        var res = base.VisitPreOptimize(node);
        Console.WriteLine($"{_padLeft}{nameof(VisitPreOptimize)}({node}) = {res}");
        return res;
    }

    protected override Expression VisitPostOptimize(Expression original, Expression optimized)
    {
        var res = base.VisitPostOptimize(original, optimized);
        Console.WriteLine($"{_padLeft}{nameof(VisitPostOptimize)}({original}, {optimized}) = {res}");
        return res;
    }

    private void Indent() => _padLeft = new string(' ', _padLeft.Length + 2);

    private void Outdent() => _padLeft = new string(' ', _padLeft.Length - 2);
}

Let's just run our logging optimizer over an expression that will have a few rewrites.

In [ ]:
var opt = new LoggingExpressionOptimizer(sem, eval);

Expression<Func<int>> f = () => "foobarqux".Substring(2, 3).ToUpperInvariant().Length + 1;

var res = opt.Visit(f);

Console.WriteLine(res);

Visit(() => ("foobarqux".Substring(2, 3).ToUpperInvariant().Length + 1)) 
{


  ShouldOptimize(() => ("foobarqux".Substring(2, 3).ToUpperInvariant().Length + 1)) = True


  VisitPreOptimize(() => ("foobarqux".Substring(2, 3).ToUpperInvariant().Length + 1)) = () => ("foobarqux".Substring(2, 3).ToUpperInvariant().Length + 1)


  Visit(("foobarqux".Substring(2, 3).ToUpperInvariant().Length + 1)) 
  {


    ShouldOptimize(("foobarqux".Substring(2, 3).ToUpperInvariant().Length + 1)) = True


...

    VisitPostOptimize(("foobarqux".Substring(2, 3).ToUpperInvariant().Length + 1), 4) = 4


  } = 4


  VisitPostOptimize(() => ("foobarqux".Substring(2, 3).ToUpperInvariant().Length + 1), () => 4) = () => 4


} = () => 4


() => 4


## More optimizations

Besides partial evaluation and constant folding, the optimizer has a lot more optimization techniques under its belt. The list is quite long, so we'll limit ourselves to exploring a few of them here.

### Branch analysis

`ConditionalExpression` can be used for the conditional ternary operator `?:` as well as `if` statements. The expression optimizer can remove such branches if the condition evaluates to a constant. For example:

In [ ]:
Expression<Func<string, char?>> getFirstChar = s => s != null && s.Length > 0 ? s[0] : null;

var expr = Expression.Invoke(getFirstChar, Expression.Constant("bar"));

Console.WriteLine(opt.Visit(expr));

Visit(Invoke(s => IIF(((s != null) AndAlso (s.Length > 0)), Convert(s.get_Chars(0), Nullable`1), null), "bar")) 
{


  ShouldOptimize(Invoke(s => IIF(((s != null) AndAlso (s.Length > 0)), Convert(s.get_Chars(0), Nullable`1), null), "bar")) = True


  VisitPreOptimize(Invoke(s => IIF(((s != null) AndAlso (s.Length > 0)), Convert(s.get_Chars(0), Nullable`1), null), "bar")) = Invoke(s => IIF(((s != null) AndAlso (s.Length > 0)), Convert(s.get_Chars(0), Nullable`1), null), "bar")


  Visit(s => IIF(((s != null) AndAlso (s.Length > 0)), Convert(s.get_Chars(0), Nullable`1), null)) 
  {


    ShouldOptimize(s => IIF(((s != null) AndAlso (s.Length > 0)), Convert(s.get_Chars(0), Nullable`1), null)) = True


...

    VisitPostOptimize(IIF((("bar" != null) AndAlso ("bar".Length > 0)), Convert("bar".get_Chars(0), Nullable`1), null), b) = b


  } = b


  VisitPostOptimize(Invoke(s => IIF(((s != null) AndAlso (s.Length > 0)), Convert(s.get_Chars(0), Nullable`1), null), "bar"), b) = b


} = b


b


A lot is going on here. Let's break it down step by step. Our original expression is:


```csharp
(s => s => s != null && s.Length > 0 ? s[0] : null)("bar")
```

First, there's beta reduction where the constant string got inlined in the lambda.

```csharp
"bar" != null && "bar".Length > 0 ? "bar"[0] : null
```

Next, the optimizer can determine that `"bar"` is never null. It knows that's the case of `new` expressions and a couple of other nodes, but also for constants it can inspect. In addition, it can ask the semantic provider whether an expression can never be null by using `IsNeverNull`.

> **Note:** The optimizer predates the work on nullability analysis that was done in C# 8.0. A future revision of the optimizer could support much more nullability checks, e.g. for the return types of methods.

Because `"bar"` can never be null, the expression can get reduced to:

```csharp
true && "bar".Length > 0 ? "bar"[0] : null
```

Rules for the binary `&&` operator enable dropping the `true` operand. (If the left operand were `false`, the expression optimizer would reduce the entire `&&` expression to `false`.) Now we have:

```csharp
"bar".Length > 0 ? "bar"[0] : null
```

Next, we're faced with a `MemberExpression` for `string.Length`, which results in a check for purity. Note there are more semantic questions asked, which we've never touched upon. In particular, the optimizer will check if the operand `IsAlwaysNull`. If so, it can directly emit code to `throw new NullReferenceException()`. Further optimizations of `TryExpression` nodes can also reason over exception flow. In this case, `"bar"` is never null, and `"bar".Length` will get evaluated to `3`. This can be seen in the output above, with a logging message indicating that the evaluator factory was consulted to build an evaluator for `(string s) => s.Length`, which then got passed `"bar"`. This results in:

```csharp
3 > 0 ? "bar"[0] : null
```

Partial evaluation of `3 > 0` results in `true`:

```csharp
true ? "bar"[0] : null
```

This in turn enables branch prediction, so we end up with:

```csharp
"bar"[0]
```

Once more, we have a pure member for the indexer of `string`, resulting in the construction of an evaluator, which finally results in:

```csharp
'b'
```

Similar optimizations exist for `SwitchExpression` nodes where a `SwitchCase` "arm" of the expression can be predicted if the test value is a constant.

### Inlining of invocations

As we've seen in the samples before, the expression optimizer also knows how to inline `InvocationExpression` nodes applied to `LambdaExpression` operands. This form of beta reduction is only carried out if it's safe to do so, i.e. if side-effects would not get reordered or dropped on the floor. Beta reduction often occurs after binding template expressions with concrete parameter values. An example is shown below.

In [ ]:
Expression<Func<IEnumerable<int>, int, IEnumerable<int>>> query = (xs, a) => xs.Where(x => x > a);

var boundQuery = Expression.Invoke(query, Expression.Constant(new[] { 1, 2, 3 }), Expression.Constant(0));

Console.WriteLine(opt.Visit(boundQuery));

Visit(Invoke((xs, a) => xs.Where(x => (x > a)), value(System.Int32[]), 0)) 
{


  ShouldOptimize(Invoke((xs, a) => xs.Where(x => (x > a)), value(System.Int32[]), 0)) = True


  VisitPreOptimize(Invoke((xs, a) => xs.Where(x => (x > a)), value(System.Int32[]), 0)) = Invoke((xs, a) => xs.Where(x => (x > a)), value(System.Int32[]), 0)


  Visit((xs, a) => xs.Where(x => (x > a))) 
  {


    ShouldOptimize((xs, a) => xs.Where(x => (x > a))) = True


...

    } = x => (x > 0)


  } = value(System.Int32[]).Where(x => (x > 0))


  VisitPostOptimize(Invoke((xs, a) => xs.Where(x => (x > a)), value(System.Int32[]), 0), value(System.Int32[]).Where(x => (x > 0))) = value(System.Int32[]).Where(x => (x > 0))


} = value(System.Int32[]).Where(x => (x > 0))


value(System.Int32[]).Where(x => (x > 0))


### Exception flow analysis

As one final example, we'll consider a more complex statement tree to illustrate how the optimizer can reason over exception flow as well.

In [ ]:
var expr =
    Expression.TryCatch(
        Expression.Convert(
            Expression.AddChecked(Expression.Constant(int.MaxValue), Expression.Constant(1)),
            typeof(int?)
        ),
        Expression.Catch(
            typeof(OverflowException),
            Expression.Default(typeof(int?))
        )
    );

Console.WriteLine(opt.Visit(expr));

Visit(try { ... }) 
{


  ShouldOptimize(try { ... }) = True


  VisitPreOptimize(try { ... }) = try { ... }


  Visit(Convert((2147483647 + 1), Nullable`1)) 
  {


    ShouldOptimize(Convert((2147483647 + 1), Nullable`1)) = True


...

  Visit() 
  {


  } = 


  VisitPostOptimize(try { ... }, default(Nullable`1)) = default(Nullable`1)


} = default(Nullable`1)


default(Nullable`1)


During optimization of the `+` operation with constant operands, partial evaluation triggered an `OverflowException`. This then caused the exception flow analysis to kick in, where the optimizer emulated the exception flow and ended up picking the `catch (OverflowException)` block, which returned a `default(int?)` expression. Even if this catch block would contain a non-trivial expression (e.g. `Console.WriteLine("Oops!")`), the optimizer would be able to eliminate the whole `TryExpression` in favor of the catch block's body. A more complex example is shown here:

In [ ]:
var expr =
    Expression.TryCatch(
        Expression.Call(
            typeof(Console).GetMethod(nameof(Console.WriteLine), new[] { typeof(int) }),
            Expression.AddChecked(Expression.Constant(int.MaxValue), Expression.Constant(1))
        ),
        Expression.Catch(
            typeof(OverflowException),
            Expression.Call(
                typeof(Console).GetMethod(nameof(Console.WriteLine), new[] { typeof(string) }),
                Expression.Constant("Oops!")
            )
        )
    );

Console.WriteLine(opt.Visit(expr));

Visit(try { ... }) 
{


  ShouldOptimize(try { ... }) = True


  VisitPreOptimize(try { ... }) = try { ... }


  Visit(WriteLine((2147483647 + 1))) 
  {


    ShouldOptimize(WriteLine((2147483647 + 1))) = True


...

  Visit() 
  {


  } = 


  VisitPostOptimize(try { ... }, WriteLine("Oops!")) = WriteLine("Oops!")


} = WriteLine("Oops!")


WriteLine("Oops!")


All of the logic that did the pure computation and triggered the exception was dropped and all that remains in a simple `Console.WriteLine("Oops!");` statement.

Obviously, this shows an extreme example of optimization where all the circumstances are just right to cause a massive reduction of the original expression. However, many of the optimization rules working in concert results in very powerful optimization capabilities.